# Fullstack GPT: #5.0 ~ #5.8

- Tasks:
    - 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
        - [x] [ConversationBufferMemory](https://python.langchain.com/v0.1/docs/modules/memory/types/buffer/) 등 강의에서 배운 메모리 중 하나를 사용하여 이전 대화 기록을 기억하고 기록을 이용한 답변을 제공할 수 있도록 합니다.
        - [x] 채팅 형식의 메모리 기록을 프롬프트에 추가하고 싶을 때는 [MessagesPlaceholder](https://python.langchain.com/v0.1/docs/modules/memory/adding_memory/#adding-memory-to-a-chat-model-based-llmchain)를 이용하세요.
        - [x] RunnablePassthrough를 활용하면 LCEL 체인을 구현할 때 메모리 적용을 쉽게 할 수 있습니다. RunnablePassthrough는 메모리를 포함한 데이터를 체인의 각 단계에 전달하는 역할을 합니다. (강의 #5.7 1:04~ 참고)
    - [x] The chain should take the title of a movie and reply with three emojis that represent the movie. (i.e "Top Gun" -> "🛩️👨‍✈️🔥". "The Godfather" -> "👨‍👨‍👦🔫🍝 ").
    - [x] 항상 세 개의 이모티콘으로 답장하도록 [FewShotPromptTemplate](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/few_shot_examples/) 또는 [FewShotChatMessagePromptTemplate](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/few_shot_examples_chat/)을 사용하여 체인에 예시를 제공하세요.
    - [x] To check that the memory is working ask the chain about two movies and then in another cell ask the chain to tell you what is the movie you asked about first.

In [1]:
examples = [
    {"question": "Top Gun", "answer": "🛩️👨‍✈️🔥"},
    {"question": "The Godfather", "answer": "👨‍👨‍👦🔫🍝"},
]

In [15]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

def load_memory(_):
    return memory.load_memory_variables({})['history']

def invoke_chain(movie_title):
    result = chain.invoke({"question": movie_title})
    memory.save_context({"input": movie_title}, {"output": result.content})

llm = ChatOpenAI(
    model="gpt-5-nano",
    temperature=1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
memory = ConversationBufferMemory(return_messages=True)

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie geek. Take the title of a movie and reply with three emojis that represent the movie."),
    MessagesPlaceholder(variable_name="history"),
    few_shot_prompt,
    ("human", "{question}"),
])

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

invoke_chain("Love Letter")
invoke_chain("K-pop Demon Hunters")

💌🌧️💖🎤👹⚔️

In [16]:
invoke_chain("What was the first movie i've asked?")

💌🌧️💖

In [ ]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

set_llm_cache(InMemoryCache())

def invoke_chain(movie_title):
    result = chain.invoke({"question": movie_title})
    memory.save_context({"input":movie_title}, {"output":result.content})

memory = ConversationBufferMemory()
llm = ChatOpenAI(
    model="gpt-5-nano",
    temperature=1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

example_prompt = PromptTemplate.from_template("""
    Human: {question},
    AI: {answer}
""")
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="{question}",
    input_variables=["question"],
)
prompt = PromptTemplate.from_template("""
    Take the title of a movie and reply with three emojis that represent the movie.
                                      
    {history}
    {few_shot_prompt}
                                      
    Human: {question}
    You:
""")

chain = (
    RunnablePassthrough.assign(
    history = lambda x: memory.load_memory_variables({})['history'],
    few_shot_prompt = lambda x: few_shot_prompt,
    )
| prompt
| llm
)

invoke_chain("DongJu")
invoke_chain("No Other Choice")

📝 ✍️ 🇰🇷❌🚪🔒

In [5]:
chain.invoke({"question": "what is the movie i asked about first looking at history?"})

DongJu

AIMessageChunk(content='DongJu')